In [75]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import json

Todo:
1. Dropdown menu
2. Borough bar chart (req 1)
3. Neighborhood bar chart (req 1,2)
4. Map of NYC (req 1, 2*/3*)
5. Scatterplots (req 1, 2*/3*)

In [76]:
# load in the data csv files
tree_df = pd.read_csv('./final_data/trees.csv')
borough_df = pd.read_csv('./final_data/boroughs.csv')
neighborhood_df = pd.read_csv('./final_data/UHF.csv')

In [77]:

with open('./geo_data/borough.geo.json', 'r') as file:
    boroughs = json.load(file)
with open('./geo_data/UHF34.geo.json', 'r') as file:
    nbds = json.load(file)
borough_df['BoroName'] = borough_df['Borough']
del nbds["features"][0]

In [78]:
# make the borough graph
fig = px.bar(y=borough_df["Borough"], x=borough_df["Number of Trees"], orientation='h')
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
borough_bar = go.Figure(data=fig)

In [79]:
# make the neighborhood graph
fig = px.bar(x=neighborhood_df["UHF34 Neighborhood Name"], y=neighborhood_df["Number of Trees"], orientation='h')
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
neighborhood_bar = go.Figure(data=fig)

In [ ]:
# TODO: make the map ???
dataname = 'Number of Trees'
fig = go.Figure(data=go.Choropleth(
        geojson=boroughs, # for UHF use `nbds`
        colorscale="Viridis",
        locations=borough_df['BoroName'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.BoroName", #for UHF use "properties.UHF" 
        z=borough_df[dataname], # for UHF use `neighborhood_df[dataname]`
        zmin=0,
        zmax=borough_df[dataname].max()
        ))

fig.update_layout(geo = dict(
        landcolor = "rgb(212, 212, 212)",
        resolution = 50,
        lonaxis = dict(
            showgrid = False,
            gridwidth = 0.5,
            range= [ -140.0, -55.0 ],
            dtick = 5
        ),
        lataxis = dict (
            showgrid = False,
            gridwidth = 0.5,
            range= [ 20.0, 60.0 ],
            dtick = 5
        )
    ),
    margin=dict(l=20, r=20, t=20, b=20)
    )

map = go.Figure(data=fig)
map.update_geos(fitbounds="locations")
()


()

In [135]:
# make the scatterplots
fig1 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Fine Particle Pollution"])
fig2 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Diameter"])
fig3 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Summer Surface Temperature"])
fig1.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig2.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig3.update_layout(margin=dict(l=20, r=20, t=20, b=20))
metric1_scatter = go.Figure(data=fig1)
metric2_scatter = go.Figure(data=fig2)
metric3_scatter = go.Figure(data=fig3)

neighborhood_df.head()

,Unnamed: 0,Average Status,Average Diameter,Borough,UHF34 Code,Number of Trees,Average Summer Surface Temperature,Average Fine Particle Pollution,Average Ozone Pollution,UHF34 Neighborhood Name
0,0,2.709569,13.385147,Brooklyn,209,14327,94.8,7.671429,31.921429,Bensonhurst - Bay Ridge
1,1,2.690632,14.122453,Queens,404406,36177,97.6,7.600000,31.471429,Bayside Little Neck-Fresh Meadows
2,2,2.705653,11.143644,Brooklyn,203,19973,98.5,8.214286,30.542857,Bedford Stuyvesant - Crown Heights
3,3,2.676278,13.192261,Brooklyn,206,21398,98.6,7.907143,31.264286,Borough Park
4,4,2.653037,14.679529,Brooklyn,208,19100,95.9,7.578571,33.800000,Canarsie - Flatlands


In [168]:
app = Dash(__name__) # TODO: add stylesheet
app.layout = html.Div([
    html.Div([
        dcc.Dropdown(
            ['Number of Trees', 'Average Status', 'Average Diameter', 'Average Fine Particle Pollution', 'Average Ozone Pollution', 'Average Summer Surface Temperature'],
            'Number of Trees',
            id='metric-dropdown'
        ),
        html.Div([
            html.Div(
                [dcc.Graph(id='borough-bar', figure=borough_bar),
                    html.Button('Deselect Borough', id='borough-deselect-button')],
                    id='borough-div',
                    className='column'
            ),
            html.Div(
                [ dcc.Graph(id='neighborhood-bar', figure=neighborhood_bar),
                html.Button('Deselect Neighborhood', id='neighborhood-deselect-button')],
                id='neighborhood-div',
                className='column'
            )],
            id='bar-div',
            className='row'
        ),
    ],
    id='far-left-div',
    className='column'
    ),
    html.Div([
        dcc.Graph(id='map', figure=map)
        ],
        id='map-div',
        className='column'
    ),
    html.Div([
            dcc.Graph(id='metric1-scatter', className='scatter', figure=metric1_scatter),
            dcc.Graph(id='metric2-scatter', className='scatter', figure=metric2_scatter),
            dcc.Graph(id='metric3-scatter', className='scatter', figure=metric3_scatter)
        ],
        id='scatter-div',
        className='column'
    )
],
className='row'
)

# OUTPUT borough-bar
# TASK 1: metric-dropdown to borough-bar - Elaine
# * generate new borough bar graph based on chosen metric
# * sort new borough bar graph by amount of metric per borough
# TODO: TASK 2: borough-deselect-button to borough-bar - Liberty
# * stop highlighting boroughs on the bar chart
@app.callback(
    Output("borough-bar", "figure"),
    [Input("metric-dropdown", "value"), 
     Input('borough-bar', 'clickData'),
     Input('borough-deselect-button', 'n_clicks')
     ]
)
def update_boroughbar_from_metricdropdown(chosen_metric, clicked_bar, borough_deselect_btn):
     # If the borough is clicked, store the clicked borough
    if clicked_bar:
        clicked_borough = clicked_bar['points'][0]['y']
    else:
        clicked_borough = None
    
    # If the deselect button is clicked, reset the selection (if the button is clicked at least once)
    if (borough_deselect_btn or 0) > 0:  # Use 0 if borough_deselect_btn is None
        clicked_borough = None

    # Create the bar chart
    fig = px.bar(
        y=borough_df["Borough"], 
        x=borough_df[chosen_metric], 
        title=f'{chosen_metric} by Borough', 
        orientation='h', 
    )

    # Color the clicked borough
    if clicked_borough:
        colors = ['purple' if borough == clicked_borough else '#636EFA' for borough in borough_df["Borough"]]
        fig.update_traces(marker_color=colors)        

    # Update axes and layout
    fig.update_yaxes(title="Borough")
    fig.update_xaxes(title=chosen_metric)
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=20, r=20, t=20, b=20))

    borough_bar = go.Figure(data=fig)
    return borough_bar

# Callback to reset the n_clicks of borough-deselect-button
@app.callback(
    Output('borough-deselect-button', 'n_clicks'),
    [Input('borough-deselect-button', 'n_clicks')]
)
def reset_borough_deselect_button(reset_clicks):
    # Reset the n_clicks of the borough-deselect-button to 0 when the reset button is clicked
    return 0

# OUTPUT borough-deselect-button
# TASK 1: reset button clicks to 0
# @app.callback(Output('borough-deselect-button','n_clicks'),
#              [Input('borough-deselect-button','n_clicks')])
# def update(input_clicks):
#     return 0

# OUTPUT neighborhood-bar
# TASK 1: metric-dropdown to neighborhood-bar - Elaine
# * generate new neighborhood bar graph based on chosen metric
# * sort new neighborhood bar graph by amount of metric per borough
# * must keep in mind if there is a borough selected---then filtering is needed. but if not, do not filter by borough
# TASK 2: borough-bar to neighborhood-bar - Liberty
# * deselect neighborhood, if selected
# * filter neighborhoods based on chosen borough
# TODO: TASK 3: borough-deslect-button to neighborhood-bar - Liberty
# * stop filtering the neighborhoods
# * stop highlighting neighborhoods, if highlighted
@app.callback(
    Output("neighborhood-bar", "figure"),
    [Input("metric-dropdown", "value"), 
     Input('borough-bar', 'clickData'),
     #Input('borough-deselect-button', 'n_clicks')
     ]
)
def update_neighborhoodbar_from_metricdropdown(chosen_metric, clicked_bar):
    clicked_borough = clicked_bar['points'][0]['y'] if clicked_bar else None
    
    # if borough_deselect_btn and borough_deselect_btn > 0:
    #     clicked_borough = None

    if clicked_borough != None:
        neighborhood_df_filtered = neighborhood_df[neighborhood_df["Borough"] == clicked_borough]
    else:
        neighborhood_df_filtered = neighborhood_df
    
    # TODO: if borough is selected, filter neighborhood_df_filtered by borough
    fig = px.bar(
        y=neighborhood_df_filtered["UHF34 Neighborhood Name"], 
        x=neighborhood_df_filtered[chosen_metric], 
        title=f'{chosen_metric} by Neighborhood', 
        orientation='h', 
    )
    fig.update_yaxes(title="Neighborhood")
    fig.update_xaxes(title=chosen_metric)
    fig.update_layout(yaxis={'categoryorder':'total ascending'}, margin=dict(l=0, r=0, t=0, b=0))
    borough_bar = go.Figure(data=fig)
    return borough_bar

# OUTPUT map
# TASK 1: metric-dropdown to map - Liberty
# * switch map coloring based on the chosen metric
# * Keep in mind if we are zoomed into a neighborhood or not (perhaps there's a way to change the map coloring without redrawing the whole map?)
# TODO: TASK 2: borough-bar to map
# * zoom into chosen borough
# TODO: TASK 3: neighborhood-bar to map
# * zoom into neighborhood
# * show tree locations as points?
# * ability to click on point and have tooltip appear with tree info
# TODO: TASK 4: borough-deselect-button to map
# * zoom out from borough
# TODO: TASK 5: neighborhood-deselect-button to map
# * zoom out to borough (or to whole NYC map, if borough is not selected)
# * stop showing individual trees
@app.callback(
    Output("map", "figure"),
    [Input("metric-dropdown", "value"), 
     Input('borough-bar', 'clickData')]
)
def update_map(chosen_metric, clicked_bar):
     # If the borough is clicked, store the clicked borough
    if clicked_bar:
        clicked_borough = clicked_bar['points'][0]['y']
    else:
        clicked_borough = None

    if clicked_borough == None:
        fig = go.Figure(data=go.Choropleth(
        geojson=boroughs, # for UHF use `nbds`
        colorscale="Viridis",
        locations=borough_df['BoroName'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.BoroName", #for UHF use "properties.UHF" 
        z=borough_df[chosen_metric], # for UHF use `neighborhood_df[dataname]`
        zmin=0,
        zmax=borough_df[chosen_metric].max()
        ))

        fig.update_layout(geo = dict(
                landcolor = "rgb(212, 212, 212)",
                resolution = 50,
                lonaxis = dict(
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ -140.0, -55.0 ],
                    dtick = 5
                ),
                lataxis = dict (
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ 20.0, 60.0 ],
                    dtick = 5
                )
            ),
            margin=dict(l=20, r=20, t=20, b=20)
            )

        map = go.Figure(data=fig)
        map.update_geos(fitbounds="locations")
        ()
    elif clicked_borough != None:
        fig = go.Figure(data=go.Choropleth(
        geojson=nbds, # for UHF use `nbds`
        colorscale="Viridis",
        locations=neighborhood_df['UHF34 Code'], #for UHF use `neighborhood_df['UHF34 Code']`
        locationmode="geojson-id",
        featureidkey="properties.UHF", #for UHF use "properties.UHF" 
        z=neighborhood_df[chosen_metric], # for UHF use `neighborhood_df[dataname]`
        zmin=0,
        zmax=neighborhood_df[chosen_metric].max()
        ))

        fig.update_layout(geo = dict(
                landcolor = "rgb(212, 212, 212)",
                resolution = 50,
                lonaxis = dict(
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ -140.0, -55.0 ],
                    dtick = 5
                ),
                lataxis = dict (
                    showgrid = False,
                    gridwidth = 0.5,
                    range= [ 20.0, 60.0 ],
                    dtick = 5
                )
            ),
            margin=dict(l=20, r=20, t=20, b=20)
        )
        map = go.Figure(data=fig)
        if clicked_borough == "Manhattan":
            map.update_geos(
            center=dict(lat=40.7685, lon=-73.9822),  
            projection_scale=175)
        elif clicked_borough =="Brooklyn":
            map.update_geos(
            center=dict(lat=40.6782, lon=-73.9442),  
            projection_scale=175)
        elif clicked_borough =="Bronx":
            map.update_geos(
            center=dict(lat=40.8448, lon=-73.8648),  
            projection_scale=250)
        elif clicked_borough =="Queens":
            map.update_geos(
            center=dict(lat=40.7282, lon=-73.7949),  
            projection_scale=110)
        elif clicked_borough =="Staten Island":
            map.update_geos(
            center=dict(lat=40.5795, lon=-74.1502),  
            projection_scale=240)

    return map


#OUTPUT metric1-scatter
# TASK 1: metric-dropdown to metric1-scatter
# * switch graph axes and data presented based on the chosen metric
# * TODO: keep in mind whether we need to highlight a single neighborhood or a borough
# TODO: TASK 2: borough-bar to metric1-scatter
# * highlight all points in the borough
# TODO: TASK 3: neighborhood-bar to metric1-scatter
# * highlight the singular point representing the neighborhood
# TODO: TASK 4: borough-deselect-button to metric1-scatter
# * stop highlighting points
# TODO: TASK 5: neighborhood-deselect-button to metric1-scatter
# * stop highlighting the neighborhood, start highlighting the whole borough (or nothing if no borough selected)
@app.callback(
    Output("metric1-scatter", "figure"),
    [Input("metric-dropdown", "value"),
    Input('borough-bar', 'clickData')]
)
def update_metric1_scatter(chosen_metric, clicked_bar):
    # Define valid scatter plots based on the chosen metric
    if clicked_bar:
        clicked_borough = clicked_bar['points'][0]['y']
        highlighted_df = neighborhood_df[neighborhood_df["Borough"] == clicked_borough]
    else:
        clicked_borough = None

    if chosen_metric == 'Number of Trees':
        fig1 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Fine Particle Pollution"])
        fig1.update_layout(xaxis_title="Number of Trees", yaxis_title="Average Fine Particle Pollution")  
        if clicked_borough:
            # Add highlighted points to the plot
            fig1.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Fine Particle Pollution"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Status':
        fig1 = px.scatter(x=neighborhood_df["Average Status"], y=neighborhood_df["Average Fine Particle Pollution"])
        fig1.update_layout(xaxis_title="Average Status", yaxis_title="Average Fine Particle Pollution") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig1.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Fine Particle Pollution"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Diameter':
        fig1 = px.scatter(x=neighborhood_df["Average Diameter"], y=neighborhood_df["Average Fine Particle Pollution"])
        fig1.update_layout(xaxis_title="Average Diameter", yaxis_title="Average Fine Particle Pollution") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig1.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Fine Particle Pollution"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Fine Particle Pollution':
        fig1 = px.scatter(x=neighborhood_df["Average Fine Particle Pollution"], y=neighborhood_df["Number of Trees"])
        fig1.update_layout(xaxis_title="Average Fine Particle Pollution", yaxis_title="Number of Trees") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig1.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Number of Trees"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Ozone Pollution':
        fig1 = px.scatter(x=neighborhood_df["Average Ozone Pollution"], y=neighborhood_df["Number of Trees"])
        fig1.update_layout(xaxis_title="Average Ozone Pollution", yaxis_title="Number of Trees") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig1.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Number of Trees"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Summer Surface Temperature':
        fig1 = px.scatter(x=neighborhood_df["Average Summer Surface Temperature"], y=neighborhood_df["Number of Trees"])
        fig1.update_layout(xaxis_title="Average Summer Surface Temperature", yaxis_title="Number of Trees") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig1.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Number of Trees"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    else:
        # Provide a default plot in case no match is found
        empty_df = pd.DataFrame({'x': [None], 'y': [None]})  # Create a DataFrame with NaN values
        fig1 = px.scatter(empty_df, x='x', y='y')  # Plot empty data

   

    fig1.update_layout(margin=dict(l=20, r=20, t=20, b=20))
    metric1_scatter = go.Figure(data=fig1)
    return metric1_scatter

#OUTPUT metric2-scatter
# TASK 1: metric-dropdown to metric2-scatter
# * switch graph axes and data presented based on the chosen metric
# * TODO: keep in mind whether we need to highlight a single neighborhood or a borough
# TODO: TASK 2: borough-bar to metric2-scatter
# * highlight all points in the borough
# TODO: TASK 3: neighborhood-bar to metric2-scatter
# * highlight the singular point representing the neighborhood
# TODO: TASK 4: borough-deselect-button to metric2-scatter
# * stop highlighting points
# TODO: TASK 5: neighborhood-deselect-button to metric2-scatter
# * stop highlighting the neighborhood, start highlighting the whole borough (or nothing if no borough selected)
@app.callback(
    Output("metric2-scatter", "figure"),
    [Input("metric-dropdown", "value"),
     Input('borough-bar', 'clickData')]
)
def update_metric1_scatter(chosen_metric, clicked_bar):
    # Define valid scatter plots based on the chosen metric
    if clicked_bar:
        clicked_borough = clicked_bar['points'][0]['y']
        highlighted_df = neighborhood_df[neighborhood_df["Borough"] == clicked_borough]
    else:
        clicked_borough = None

    # Define valid scatter plots based on the chosen metric
    if chosen_metric == 'Number of Trees':
        fig2 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Ozone Pollution"])
        fig2.update_layout(xaxis_title="Number of Trees", yaxis_title="Average Ozone Pollution")
        if clicked_borough:
            # Add highlighted points to the plot
            fig2.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Ozone Pollution"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Status':
        fig2 = px.scatter(x=neighborhood_df["Average Status"], y=neighborhood_df["Average Ozone Pollution"])
        fig2.update_layout(xaxis_title="Average Status", yaxis_title="Average Ozone Pollution") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig2.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Ozone Pollution"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Diameter':
        fig2 = px.scatter(x=neighborhood_df["Average Diameter"], y=neighborhood_df["Average Ozone Pollution"])
        fig2.update_layout(xaxis_title="Average Diameter", yaxis_title="Average Ozone Pollution") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig2.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Ozone Pollution"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Fine Particle Pollution':
        fig2 = px.scatter(x=neighborhood_df["Average Fine Particle Pollution"], y=neighborhood_df["Average Status"])
        fig2.update_layout(xaxis_title="Average Fine Particle Pollution", yaxis_title="Average Status") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig2.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Status"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Ozone Pollution':
        fig2 = px.scatter(x=neighborhood_df["Average Ozone Pollution"], y=neighborhood_df["Average Status"])
        fig2.update_layout(xaxis_title="Average Ozone Pollution", yaxis_title="Average Status") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig2.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Status"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Summer Surface Temperature':
        fig2 = px.scatter(x=neighborhood_df["Average Summer Surface Temperature"], y=neighborhood_df["Average Status"])
        fig2.update_layout(xaxis_title="Average Summer Surface Temperature", yaxis_title="Average Status") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig2.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Status"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    else:
        # Provide a default plot in case no match is found
        empty_df = pd.DataFrame({'x': [None], 'y': [None]})  # Create a DataFrame with NaN values
        fig2 = px.scatter(empty_df, x='x', y='y')  # Plot empty data

    fig2.update_layout(margin=dict(l=20, r=20, t=20, b=20))
    metric2_scatter = go.Figure(data=fig2)
    return metric2_scatter

# OUTPUT metric3-scatter
# TASK 1: metric-dropdown to metric3-scatter
# * switch graph axes and data presented based on the chosen metric
# * TODO: keep in mind whether we need to highlight a single neighborhood or a borough
# TODO: TASK 2: borough-bar to metric3-scatter
# * highlight all points in the borough
# TODO: TASK 3: neighborhood-bar to metric3-scatter
# * highlight the singular point representing the neighborhood
# TODO: TASK 4: borough-deselect-button to metric3-scatter
# * stop highlighting points
# TODO: TASK 5: neighborhood-deselect-button to metric3-scatter
# * stop highlighting the neighborhood, start highlighting the whole borough (or nothing if no borough selected)
@app.callback(
    Output("metric3-scatter", "figure"),
    [Input("metric-dropdown", "value"),
     Input('borough-bar', 'clickData')]
)
def update_metric1_scatter(chosen_metric, clicked_bar):
    # Define valid scatter plots based on the chosen metric
    if clicked_bar:
        clicked_borough = clicked_bar['points'][0]['y']
        highlighted_df = neighborhood_df[neighborhood_df["Borough"] == clicked_borough]
    else:
        clicked_borough = None

    # Define valid scatter plots based on the chosen metric
    if chosen_metric == 'Number of Trees':
        fig3 = px.scatter(x=neighborhood_df["Number of Trees"], y=neighborhood_df["Average Summer Surface Temperature"])
        fig3.update_layout(xaxis_title="Number of Trees", yaxis_title="Average Summer Surface Temperature")  
        if clicked_borough:
            # Add highlighted points to the plot
            fig3.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Summer Surface Temperature"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Status':
        fig3 = px.scatter(x=neighborhood_df["Average Status"], y=neighborhood_df["Average Summer Surface Temperature"])
        fig3.update_layout(xaxis_title="Average Status", yaxis_title="Average Summer Surface Temperature") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig3.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Summer Surface Temperature"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Diameter':
        fig3 = px.scatter(x=neighborhood_df["Average Diameter"], y=neighborhood_df["Average Summer Surface Temperature"])
        fig3.update_layout(xaxis_title="Average Diameter", yaxis_title="Average Summer Surface Temperature") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig3.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Summer Surface Temperature"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Fine Particle Pollution':
        fig3 = px.scatter(x=neighborhood_df["Average Fine Particle Pollution"], y=neighborhood_df["Average Diameter"])
        fig3.update_layout(xaxis_title="Average Fine Particle Pollution", yaxis_title="Average Diameter") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig3.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Diameter"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Ozone Pollution':
        fig3 = px.scatter(x=neighborhood_df["Average Ozone Pollution"], y=neighborhood_df["Average Diameter"])
        fig3.update_layout(xaxis_title="Average Ozone Pollution", yaxis_title="Average Diameter") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig3.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Diameter"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    elif chosen_metric == 'Average Summer Surface Temperature':
        fig3 = px.scatter(x=neighborhood_df["Average Summer Surface Temperature"], y=neighborhood_df["Average Diameter"])
        fig3.update_layout(xaxis_title="Average Summer Surface Temperature", yaxis_title="Average Diameter") 
        if clicked_borough:
            # Add highlighted points to the plot
            fig3.add_scatter(
                x=highlighted_df[chosen_metric],
                y=highlighted_df["Average Diameter"],  
                mode='markers',
                marker=dict(color='red'),
                name=f"{clicked_borough}"
            )
    else:
        # Provide a default plot in case no match is found
        empty_df = pd.DataFrame({'x': [None], 'y': [None]})  # Create a DataFrame with NaN values
        fig3 = px.scatter(empty_df, x='x', y='y')  # Plot empty data

    if clicked_borough:
        # Filter the DataFrame based on the clicked borough
        highlighted_df = neighborhood_df[neighborhood_df["Borough"] == clicked_borough]
        
        # Add highlighted points to the plot
        fig3.add_scatter(
            x=highlighted_df[chosen_metric],
            y=highlighted_df["Average Fine Particle Pollution"],  # Adjust this based on your data
            mode='markers',
            marker=dict(color='red'),
            name=f"{clicked_borough}"
        )
    fig3.update_layout(margin=dict(l=20, r=20, t=20, b=20))
    metric3_scatter = go.Figure(data=fig3)
    return metric3_scatter

if __name__ == '__main__':
    app.run_server(debug=True)